In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
connection_uri = (
    "mssql+pyodbc://TEDPHO0134\TANDT_DP/DP_Tan?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)

<h4>1. Lấy bảng Commision <h4>

In [2]:
cutoff = '202401'
commission_query = f"select * from AGENT_COM_CUTOFF where CUTOFF like '{cutoff}%'"
agent_com = pd.read_sql(commission_query, conn)

<h4>2. Lấy bảng PY2 <h4>

In [9]:
py2_query = f"select * from AGENT_COM_CUTOFF where CUTOFF like '{cutoff}%'"
py2 = pd.read_sql(py2_query, conn)

<h4>3. Copy bảng <h4>

In [3]:
com_table = agent_com
#py2_table = py2

In [4]:
com_table.head(10)

,Policy No,Product Code,Premium transaction,Premium transaction name,Premium Collected,Collected Date,Applied Premium Date,FYC,RYC,Shared Commission Rate,...,Servicing Agent,Commission Agent,Transfer Date,Proposal Receive Date,Freelook,RISK_COMMENCE_DATE,Receive Policy date,Premium Year,CUTOFF,RETURN_DATE
0,80001330,EN04,B522,Premium Collection,7449000.0,2017-12-29,2017-12-29,0.0,745000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,3,20240115,None
1,80001330,EN04,B522,Premium Collection,7449000.0,2019-01-28,2019-01-28,0.0,745000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,4,20240115,None
2,80001330,EN04,B522,Premium Collection,7449000.0,2020-02-17,2020-02-17,0.0,745000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,5,20240115,None
3,80001330,EN04,B522,Premium Collection,7449000.0,2020-12-21,2020-12-21,0.0,745000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,6,20240115,None
4,80001330,EN04,B522,Premium Collection,7449000.0,2021-12-21,2021-12-21,0.0,149000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,7,20240115,None
5,80001330,EN04,B522,Premium Collection,7449000.0,2022-12-31,2022-12-31,0.0,149000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,8,20240115,None
6,80001330,EN04,B522,Premium Collection,7449000.0,2024-01-08,2024-01-08,0.0,149000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,9,20240115,None
7,80001331,EN04,T642,Contract Issue,7523000.0,2015-12-23,2015-12-23,2708000.0,0.0,100.0,...,60001059,60001059,2015-12-23,2015-12-23,2943.0,2015-12-21,2015-12-26,1,20240115,None
8,80001331,EN04,B522,Premium Collection,7523000.0,2016-12-21,2016-12-21,0.0,752000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,2,20240115,None
9,80001331,EN04,B522,Premium Collection,7523000.0,2017-12-29,2017-12-29,0.0,752000.0,100.0,...,60001470,60001470,2016-01-14,2015-12-23,2943.0,2015-12-21,2015-12-26,3,20240115,None


<h4>4. Bổ sung cột liên quan đến CUTOFF <h4>

In [5]:
#Thêm cột ngày xét
Monthly_Com = com_table
Monthly_Com['Date_On_Set'] = "" #Tạo ra cột trống
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[:4] ,sep ="")
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[4:6] ,sep ="-")
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[6:8] ,sep ="-")

In [6]:
#Thêm cột ưu tiên
Monthly_Com['Priority'] = Monthly_Com['CUTOFF'].rank(method='dense', ascending=True)

<h4>5. Biến cột Applied Date và Date_On_Set thành kiểu date <h4>

In [9]:
#Phải biến thành kiểu datetime trước
Monthly_Com['Date_On_Set'] = pd.to_datetime(Monthly_Com['Date_On_Set'])
#Thêm cột ngày đầu tháng:
#-> Đưa vào ngày 1 của tháng
if not('20240229' in Monthly_Com['CUTOFF'].values): 
    Monthly_Com['Start_Month'] = (Monthly_Com['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
else:
    Monthly_Com.loc[:,'Start_Month'] = '2024-01-27'

In [10]:
#Phải biến thành kiểu datetime trước
Monthly_Com['Start_Month'] = pd.to_datetime(Monthly_Com['Start_Month'])
Monthly_Com['Applied Premium Date'] = pd.to_datetime(Monthly_Com['Applied Premium Date'])

In [11]:
Monthly_Com['CheckTer'] = np.where(Monthly_Com['Terminated Date'].notnull(),"Terminated", "")

<h4> Tính theo full bảng <h4>

In [12]:
calculate_condition = (Monthly_Com['Applied Premium Date'] >= Monthly_Com['Start_Month']) &\
(Monthly_Com['Applied Premium Date'] <= Monthly_Com['Date_On_Set']) &\
( (Monthly_Com['Policy Status'] != 'SU') | (Monthly_Com['Policy Status'] != 'TR'))
calculate_month = Monthly_Com.loc[calculate_condition]
calculate_month = calculate_month.groupby(['Commission Agent','Policy No', 'Premium transaction name', 'Policy Status', 'Applied Premium Date',\
             "CheckTer",'Issued Date', 'Receive Policy date', 'Freelook', 'CUTOFF'])[['Premium Collected','FYC','RYC']].sum()
calculate_month = calculate_month.reset_index()

In [13]:
#1. Chưa pass FL
#2. Chưa có ACK
#3. Không pass FL trong 21 ngày
#4. Ter trước khi phát sinh phí
#5. Thanh toán
#len của CheckTer >1 => Terminated
calculate_month['Status'] = np.where(calculate_month['Freelook']<=21, "1", 
                     np.where(calculate_month['Receive Policy date'].isnull(), "2",
                     np.where((calculate_month['Policy Status'] == "FL") & (calculate_month['CheckTer'].str.len() <1), "3",
                     np.where((calculate_month['Policy Status'] != "FL") & (calculate_month['CheckTer'].str.len() >1), "4", 
                     np.where((calculate_month['Policy Status'] != "FL") & (calculate_month['CheckTer'].str.len() <1) & (calculate_month['Freelook']>21), "5","")     
                             ))))

In [14]:
calculate_month

,Commission Agent,Policy No,Premium transaction name,Policy Status,Applied Premium Date,CheckTer,Issued Date,Receive Policy date,Freelook,CUTOFF,Premium Collected,FYC,RYC,Status
0,60000194,80001092,Premium Collection,IF,2024-01-25,,2015-11-27,2015-12-01,2979.0,20240126,10393000.0,0.0,212000.0,5
1,60000194,80001673,Premium Collection,IF,2024-01-18,,2016-01-19,2016-01-25,2924.0,20240126,5724000.0,0.0,126000.0,5
2,60000288,80001249,Premium Collection,IF,2024-01-12,,2015-12-03,2015-12-09,2960.0,20240115,8571000.0,0.0,177000.0,5
3,60000288,80001249,Premium Collection,IF,2024-01-12,,2015-12-03,2015-12-09,2971.0,20240126,8571000.0,0.0,177000.0,5
4,60000288,80001250,Premium Collection,IF,2024-01-12,,2015-12-03,2015-12-09,2960.0,20240115,8995000.0,0.0,185000.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5145,60082266,80123526,Premium Collection,IF,2024-01-16,,2022-10-27,2022-11-10,443.0,20240126,7591000.0,0.0,767000.0,5
5146,60082770,80156501,Freelook Cancellation,FL,2024-01-05,,2023-11-30,2023-12-15,21.0,20240115,-18953000.0,-6065000.0,0.0,1
5147,60082770,80156501,Freelook Cancellation,FL,2024-01-05,,2023-11-30,2023-12-15,21.0,20240126,-18953000.0,-6065000.0,0.0,1
5148,60082770,80156502,Freelook Cancellation,FL,2024-01-04,,2023-11-30,2023-12-15,20.0,20240115,-19808000.0,-3565000.0,0.0,1


<h4>6. Xét các TH Kỳ <h4>

In [15]:
Filter_Month = calculate_month[['Commission Agent', 'Policy No', 'Premium transaction name', 'Policy Status', 'Applied Premium Date',
 'CheckTer', 'Issued Date', 'Receive Policy date', 'CUTOFF', 'Premium Collected', 'FYC', 'RYC', 'Status']]

In [16]:
Filter_Month['Mark_Time'] =  Filter_Month.groupby('Policy No')['CUTOFF'].rank(method="dense", ascending=True)

In [17]:
#Lấy các dòng cần giữ 2 dòng
keep_two = Filter_Month[(Filter_Month['Status'] == "1") | (Filter_Month['Status'] == "2") ]

In [18]:
#Lấy các dòng cần giữ 1 dòng
keep_one = Filter_Month[(Filter_Month['Status'] == "3") | (Filter_Month['Status'] == "4") | (Filter_Month['Status'] == "5") ]
keep_one = keep_one[keep_one['Mark_Time'] == 1]
keep_one

,Commission Agent,Policy No,Premium transaction name,Policy Status,Applied Premium Date,CheckTer,Issued Date,Receive Policy date,CUTOFF,Premium Collected,FYC,RYC,Status,Mark_Time
0,60000194,80001092,Premium Collection,IF,2024-01-25,,2015-11-27,2015-12-01,20240126,10393000.0,0.0,212000.0,5,1.0
1,60000194,80001673,Premium Collection,IF,2024-01-18,,2016-01-19,2016-01-25,20240126,5724000.0,0.0,126000.0,5,1.0
2,60000288,80001249,Premium Collection,IF,2024-01-12,,2015-12-03,2015-12-09,20240115,8571000.0,0.0,177000.0,5,1.0
4,60000288,80001250,Premium Collection,IF,2024-01-12,,2015-12-03,2015-12-09,20240115,8995000.0,0.0,185000.0,5,1.0
6,60000449,80003490,Premium Collection,IF,2024-01-16,,2016-06-22,2016-06-28,20240126,3122000.0,0.0,65000.0,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5139,60080745,80151504,Freelook Cancellation,FL,2024-01-18,,2023-06-30,2023-08-30,20240126,-25266000.0,-9348000.0,0.0,3,1.0
5140,60082085,80154606,Freelook Cancellation,FL,2024-01-18,,2023-07-31,2023-08-30,20240126,-29821000.0,-10390000.0,0.0,3,1.0
5141,60082085,80154620,Freelook Cancellation,FL,2024-01-18,,2023-07-31,2023-08-30,20240126,-25841000.0,-9561000.0,0.0,3,1.0
5142,60082085,80154633,Freelook Cancellation,FL,2024-01-18,,2023-07-31,2023-08-30,20240126,-25884000.0,-9602000.0,0.0,3,1.0


In [19]:
full_modified = pd.concat([keep_one, keep_two])

In [20]:
full_modified = full_modified[['Commission Agent', 'Policy No', 'Premium transaction name',
       'Policy Status', 'Applied Premium Date', 'Issued Date',
       'Receive Policy date', 'CUTOFF', 'Premium Collected', 'FYC', 'RYC', 'Mark_Time',
       'Status']]

In [21]:
#Rút ra cột Final FYC để về 0 trong các trạng thái không thanh toán
full_modified.loc[(full_modified['Status'].isin(['1','2','3','4'])), 'Final_FYC'] = "0"
full_modified.loc[(full_modified['Status'].isin(['5'])), 'Final_FYC'] = full_modified['FYC']                 

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\4093032915.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_modified.loc[(full_modified['Status'].isin(['1','2','3','4'])), 'Final_FYC'] = "0"


In [22]:
#Rút ra cột Final RYC để về 0 trong các trạng thái không thanh toán
full_modified.loc[(full_modified['Status'].isin(['1','2','3','4'])), 'Final_RYC'] = "0"
full_modified.loc[(full_modified['Status'].isin(['5'])), 'Final_RYC'] = full_modified['RYC'] 

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\4270509618.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_modified.loc[(full_modified['Status'].isin(['1','2','3','4'])), 'Final_RYC'] = "0"


In [23]:
#Tính Total Commision
full_modified['Total_Commision'] = full_modified['Final_FYC'] + full_modified['Final_RYC']

<h4>7. Xác định các kỳ thanh toán <h4>

In [24]:
Final_Com = full_modified

In [25]:
Final_Com.loc[:,'Kỳ Tính Toán'] = Final_Com['CUTOFF']
Final_Com.loc[:,'Kỳ Khóa Sổ'] = Final_Com['CUTOFF']
Final_Com.loc[(Final_Com['Status'] == "5"),'Kỳ Thanh Toán'] = Final_Com['CUTOFF']

In [26]:
Final_Com.to_csv("fullmonth.csv", encoding = 'utf-8-sig')

In [27]:
Final_Com.loc[(Final_Com['Status'] == "1"),'Note'] = "Chưa pass FL 21 ngày"
Final_Com.loc[(Final_Com['Status'] == "2"),'Note'] = "Chưa có ACK"
Final_Com.loc[(Final_Com['Status'] == "3"),'Note'] = "Không pass FL 21 ngày"
Final_Com.loc[(Final_Com['Status'] == "4"),'Note'] = "Ter trước khi phát sinh phí"
Final_Com.loc[(Final_Com['Status'] == "5"),'Note'] = "Thanh toán"

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\1287773080.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Chưa pass FL 21 ngày' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Final_Com.loc[(Final_Com['Status'] == "1"),'Note'] = "Chưa pass FL 21 ngày"


In [28]:
Final_Com.loc[(Final_Com['Final_FYC'] !=0),'FYC/RYC'] = "FYC"
Final_Com.loc[(Final_Com['Final_RYC'] !=0),'FYC/RYC'] = "RYC"

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\16362111.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'FYC' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Final_Com.loc[(Final_Com['Final_FYC'] !=0),'FYC/RYC'] = "FYC"


<h4> 8. Xét WCC <h4>

In [29]:
wcc_agent_query = f"select * from WCC_Agent where CUTOFF like '{cutoff}%'"
wcc_agent = pd.read_sql(wcc_agent_query, conn)
######
wcc_policy_query = f"select * from WCC_Policy where CUTOFF like '{cutoff}%'"
wcc_policy = pd.read_sql(wcc_policy_query, conn)

In [30]:
agent_wcc = wcc_agent
policy_wcc = wcc_policy

In [31]:
#Xét điều kiện bảng AgentWCC
agent_wcc.loc[(agent_wcc['Result'] == "Không gặp"), "Agent_Result"] = "Fail"
agent_wcc.loc[(agent_wcc['Result'] != "Không gặp"), "Agent_Result"] = "Success"

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\2569655088.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Fail' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  agent_wcc.loc[(agent_wcc['Result'] == "Không gặp"), "Agent_Result"] = "Fail"


In [32]:
#Xét điều kiên bảng PolicyWCC
policy_wcc.loc[(policy_wcc['finalResult'] == "WCC không thành công"), "Policy_Result"] = "Fail"
policy_wcc.loc[(policy_wcc['finalResult'] != "WCC không thành công"), "Policy_Result"] = "Success"

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\643089592.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Fail' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  policy_wcc.loc[(policy_wcc['finalResult'] == "WCC không thành công"), "Policy_Result"] = "Fail"


In [38]:
Full_Com = Final_Com

In [40]:
#Đổi sang kiểu int để join bảng
agent_wcc['Agent_Number'] = agent_wcc['Agent_Number'].astype('int')
Full_Com['Commission Agent'] = Full_Com['Commission Agent'].astype('int')
agent_wcc.rename(columns ={'CUTOFF': 'CUTOFF_x'}, inplace = True)

In [41]:
Full_Com = pd.merge(Full_Com, agent_wcc, left_on = 'Commission Agent', right_on = 'Agent_Number' ,how = 'left')

In [42]:
Full_Com = Full_Com[['Commission Agent', 'Policy No', 'Premium transaction name',
       'Policy Status', 'Applied Premium Date', 'Issued Date',
       'Receive Policy date', 'CUTOFF', 'Premium Collected', 'FYC', 'RYC',
       'Mark_Time', 'Status', 'Final_FYC', 'Final_RYC', 'Total_Commision',
       'Kỳ Tính Toán', 'Kỳ Khóa Sổ', 'Kỳ Thanh Toán', 'Note', 'FYC/RYC', 'Agent_Number' ,'Agent_Result' ]]

In [45]:
# Đổi sang kiểu int để join bảng
policy_wcc['CONTRACT_NUMBER'] = policy_wcc['CONTRACT_NUMBER'].astype('int')
Full_Com['Policy No'] = Full_Com['Policy No'].astype('int')
policy_wcc.rename(columns ={'CUTOFF': 'CUTOFF_x'}, inplace = True)

In [46]:
Full_Com = pd.merge(Full_Com, policy_wcc, left_on = 'Policy No', right_on = 'CONTRACT_NUMBER' ,how = 'left')

In [49]:
Full_Com= Full_Com[['Commission Agent', 'Policy No', 'Premium transaction name',
       'Policy Status', 'Applied Premium Date', 'Issued Date',
       'Receive Policy date', 'CUTOFF', 'Premium Collected', 'FYC', 'RYC',
       'Mark_Time', 'Status', 'Final_FYC', 'Final_RYC', 'Total_Commision',
       'Kỳ Tính Toán', 'Kỳ Khóa Sổ', 'Kỳ Thanh Toán', 'Note', 'FYC/RYC',
       'Agent_Number', 'Agent_Result','CONTRACT_NUMBER' ,'Policy_Result']]

In [82]:
Full_Com['Final_Result'] = np.where( (Full_Com['Agent_Result'] == 'Success') & (Full_Com['Policy_Result'] == 'Success'), Full_Com['Status'],
np.where((Full_Com['Agent_Result'] == 'Fail') | (Full_Com['Policy_Result'] == 'Fail'), "6",Full_Com['Status']))

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\1789196318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Full_Com['Final_Result'] = np.where( (Full_Com['Agent_Result'] == 'Success') & (Full_Com['Policy_Result'] == 'Success'), Full_Com['Status'],


In [88]:
Full_Com.loc[(Full_Com['Final_Result'] == "1"),'Note'] = "Chưa pass FL 21 ngày"
Full_Com.loc[(Full_Com['Final_Result'] == "2"),'Note'] = "Chưa có ACK"
Full_Com.loc[(Full_Com['Final_Result'] == "3"),'Note'] = "Không pass FL 21 ngày"
Full_Com.loc[(Full_Com['Final_Result'] == "4"),'Note'] = "Ter trước khi phát sinh phí"
Full_Com.loc[(Full_Com['Final_Result'] == "5"),'Note'] = "Thanh toán"
Full_Com.loc[(Full_Com['Final_Result'] == "6"),'Note'] = "Không hoàn thành WCC"

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\821958627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Full_Com.loc[(Full_Com['Final_Result'] == "1"),'Note'] = "Chưa pass FL 21 ngày"
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_3352\821958627.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Chưa pass FL 21 ngày' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Full_Com.loc[(Full_Com['Final_Result'] == "1"),'Note'] = "Chưa pass FL 21 ngày"
